## Tensorflow feature_column API

In [1]:
import tensorflow as tf
from sklearn.metrics import roc_auc_score,log_loss
from DeepFM_use_generator_gpu_fieldMerge import DeepFM as DeepFM_fieldMerge
from DeepFM_use_generator_gpu import DeepFM
import time
import sys
import itertools
from progressbar import ProgressBar
import numpy as np
from functools import wraps
import os
import shutil
from collections import deque
from IPython.core.interactiveshell import InteractiveShell
from functools import reduce
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
InteractiveShell.ast_node_interactivity = "all"

# 禁用GPU
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

## 获取feature_map、info.json 以及一个简易的generator

In [ ]:
def gen(p):
    with open(p,"r+") as f:
        for l in f:
            yield l.strip("[]\n")
        
data_g = gen("/home/zhoutong/data/apus_ad/data_2018-09-21_to_2018-09-26/2018-09-21_to_2018-09-25_shuffled")

base_p = "/home/zhoutong/data/apus_ad/data_2018-09-21_to_2018-09-26"
with open(base_p + "/feature_map_2018-09-21_to_2018-09-25","r+") as f: 
    feature_map = {x.strip().split("\t")[1] : x.strip().split("\t")[0] for x in f.readlines()}
with open(base_p + "/info.json", "r+") as f:
    info = eval(f.readlines()[0].strip())
all_fields = set(map(lambda x:x.split("=")[0], feature_map.values()))


## TFRecord | 完整Demo（pure python)
   - 构造TFRecord
   - 持久化存储
   - 读取解析

In [ ]:
sample_label = int([i.split(" ") for i in next(data_g).split("\t")][0][0])
sample_features_raw = [i.split(" ") for i in next(data_g).split("\t")][1:]
###########
# 事前准备： 把libsvm格式换成 [feature_name, 1.0]
sample_features = []
for features in sample_features_raw:
    print("*" * 10)
    for pair in features:
        k,v = pair.split(":")
        sample_features.append([feature_map[k],float(v)])
    print(features)

numeric_fieldList = ["ad_info__budget_unit","apolo","gama"]
multi_hot_fieldList = ["user_profile_app__install_app_list","user_profile_app__tag_unit"]


# 主要目的还是把分散的multi-hot特征 k=v1, k=v2, k=v3 合并成 k=[v1,v2,v3]
data_dict = {}
for feature_name, value in sample_features:
    if any([field in feature_name for field in numeric_fieldList]):
        # numeric_feature
        data_dict[feature_name] = value
    elif any([field in feature_name for field in multi_hot_fieldList]):
        # multi_hot_feature
        field,value_ = feature_name.split("=")
        if field not in data_dict.keys():
            data_dict[field] = [value_]
        else:
            data_dict[field] = data_dict[field] + [value_]
    else:
        field,value_ = feature_name.split("=")
        data_dict[field] = value_

##############
# 构造TFRecord
feature_dict = {}
for k,v in data_dict.items():
    if type(v)==float:
        feature_dict[k] = tf.train.Feature(float_list = tf.train.FloatList(value=[v]))
    elif type(v)==list:
        feature_dict[k] = tf.train.Feature(bytes_list = tf.train.BytesList(value=[i.encode('utf-8') for i in v]))
    else:
        feature_dict[k] = tf.train.Feature(bytes_list = tf.train.BytesList(value=[str(v).encode('utf-8')]))

features = tf.train.Features(feature=feature_dict)
example = tf.train.Example(features=features)




##############
# 持久化TFRecord
with tf.python_io.TFRecordWriter('/home/zhoutong/sample1.tfrecord') as writer:
    writer.write(example.SerializeToString())

    
    
##############
# 读取TFRecord
reader = tf.TFRecordReader()
filename_queue = tf.train.string_input_producer(['/home/zhoutong/sample1.tfrecord'])
_, serialized_example = reader.read(filename_queue)

read_feature = {}
for field in all_fields:
    if any([num_f in field for num_f in numeric_fieldList]):
        read_feature[field] = tf.FixedLenFeature([], dtype=tf.float32)
    elif any([multi_f in field for multi_f in multi_hot_fieldList]):
        read_feature[field] = tf.VarLenFeature(dtype=tf.string)
    else:
        read_feature[field] = tf.FixedLenFeature([], dtype=tf.string)

tf.train.start_queue_runners(sess)
read_data = tf.parse_single_example(serialized=serialized_example, features=read_feature)
for name, tensor in read_data.items():
    print("%s : %s" % (name, tensor.eval()))



## TFReocrd | Spark+TensorFlow
- Spark 直接生成 TFRecord 形式的文件
- TF三种方式解析TFRecord
    - DataSet
    - TFRecordReader  (<font style="color:rgb(199,21,133)">deprecated</font>)
    - tf_record_iterator

注意如果Spark没有处理null值问题，这里使用的时候，需要给 ```tf.FixedLenFeature```(或```tf.VarLenFeature```) 设置默认值参数 ```default_value=```

In [ ]:
# 获取 tfrecords 文件
# tfrecordFiles=["/home/zhoutong/data/part-total"]
tfrecordFiles=["/home/zhoutong/data/part-r-00000"]

- ### TFRecord | DataSet解析

In [ ]:
##################
# success
# 方案一： 使用 TF DataSet
def _parse_function(example_proto):
    all_fields=["cpu_cores", "client_id", "manufacturer", "network_type", "city", "memory_total", "country", "model", "sdk_level", "storage_total", "is_limit_ad_tracking", "height", "locale", "os_version", "width", "is_root", "dpi"]
    feature_structure={i:tf.FixedLenFeature([], dtype=tf.string, default_value="N/A") for i in all_fields}
    parsed_features = tf.parse_single_example(example_proto, feature_structure)
    return list(parsed_features.items())
dataset = tf.data.TFRecordDataset(tfrecordFiles)
dataset = dataset.map(_parse_function)

# iterator = dataset.make_one_shot_iterator()
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()
#############
# sess 运行
with tf.Session() as sess:
    # 除了one_shot_iterator其他类型的iterator需要初始化
    sess.run(iterator.initializer)
    for _ in range(3):
        print(type(sess.run(next_element)),":")
        print(sess.run(next_element),"\n")

- ### TFRecord | TFRecordReader

In [ ]:
########################
# fail
# 方案二： 使用以前的TFRecordReader
filename_queue = tf.train.string_input_producer(tfrecordFiles)
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)
batch = tf.train.batch([serialized_example],batch_size=5)
# 给出特征结构（这里全都是定长特征）
all_fields=["cpu_cores", "client_id", "manufacturer", "network_type", "city", "memory_total", "country", "model", "sdk_level", "storage_total", "is_limit_ad_tracking", "height", "locale", "os_version", "width", "is_root", "dpi"]
read_feature={i:tf.FixedLenFeature([], dtype=tf.string, default_value="N/A") for i in all_fields}
# 解析
# read_data = tf.parse_single_example(serialized=batch, features=read_feature)
read_data = tf.parse_example(serialized=batch, features=read_feature)
len(read_data.items())

#############
# sess 运行
# 未知错误，在sess.run(tensor)时会卡主
# with tf.Session() as sess:
#     for name, tensor in read_data.items():
#         print("%s : %s" % (name, tensor))
#         print("%s : %s" % (name, sess.run(tensor)))

- ### TFRecord | tf_record_iterator

In [ ]:
########################
# success
# 方案三： 使用 tf_record_iterator
example = tf.train.Example()
all_fields=["cpu_cores", "client_id", "manufacturer", "network_type", "city", "memory_total", "country", "model", "sdk_level", "storage_total", "is_limit_ad_tracking", "height", "locale", "os_version", "width", "is_root", "dpi"]

record_iter = tf.python_io.tf_record_iterator(tfrecordFiles[0])
for record in itertools.islice(record_iter,0,5):
    example.ParseFromString(record)
    f = example.features.feature
    feature_list = []
    for field in all_fields:
        feature_list.append([field, f[field].bytes_list.value])
    print(feature_list)

########################
# 涉及到压缩格式的TFRecord
example = tf.train.Example()
tfreocrd_option = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)
record_iter = tf.python_io.tf_record_iterator(tfrecord_file,tfreocrd_option)
for record in itertools.islice(record_iter,0,2):
    example.ParseFromString(record)
    f = example.features.feature
    feature_list = []
    for field in global_all_fields:
        feature_list.append([field, f[field].int64_list.value])
    print(feature_list)

## 使用tf.feature_column

In [ ]:
tf.feature_column.categorical_column_with_identity

tf.feature_column.categorical_column_with_vocabulary_list(key='ad_info__app_package_name',
                                                         vocabulary_list=[])

## apus_ad | 数据准备（通用）

In [ ]:
###########
# 迭代生成

class DataGenerator(object):
    def __init__(self, train_path, valid_path, max_numeric):
        self.train_path = train_path
        self.valid_path = valid_path
        self.max_numeric = max_numeric
    @staticmethod
    def print_t(param):
        sys.stdout.flush()
        now = time.strftime("|%Y-%m-%d %H:%M:%S| ", time.localtime(time.time()))
        new_params = now + ": " + param
        print(new_params)
        sys.stdout.flush()
    def get_apus_ad_train_generator(self):
        return self._yield_apus_ad_generator(self.train_path,self.max_numeric)
    def get_apus_ad_valid(self):
        return self._yield_apus_ad_generator(self.valid_path,self.max_numeric)
    @staticmethod
    def _yield_apus_ad_generator(reader_path,max_numeric):
        with open(reader_path, "r") as f:
            for line in f:
                info = line.strip("[]\n").split("\t")
                label = int(info[0])
                numeric_f = info[1].split(" ")
                category_f = info[2].split(" ")
                multi_hot_f = [x.split(" ")for x in info[3:]]

                y = [label]
                def get_idx_and_value(feature_info):
                    index = [int(x.split(":")[0]) for x in feature_info]
                    value = [float(x.split(":")[1]) for x in feature_info]
                    return index,value
                Xi_numeric, Xv_numeric = get_idx_and_value(numeric_f)
                Xi_category, Xv_category = get_idx_and_value(category_f)
                multi_hot_f_tmp = [get_idx_and_value(feature) for feature in multi_hot_f]
                Xi_multi_hot_list = [feature_idx for feature_idx,value in multi_hot_f_tmp]
                Xv_multi_hot_list = [value for feature_idx,value in multi_hot_f_tmp]
                # Xv_numeric 归一化
                if len(max_numeric)>0:
                    # 根据索引是0还是1还是2从 max_nuemeric里面取值
                    Xv_numeric = []
                    for i in numeric_f:
                        idx = int(i.split(":")[0])
                        v = float(i.split(":")[1])
                        Xv_numeric.append(v/max_numeric[idx])
                    Xv_numeric = list(map(lambda x: x if x<=1 else 1, Xv_numeric))
                yield [y, Xi_numeric, Xv_numeric, Xi_category, Xv_category, Xi_multi_hot_list, Xv_multi_hot_list]


##########
# 数据准备：两个多值离散特征
train_double_multi = "/home/zhoutong/data/apus_ad/data_2018-09-03_to_2018-09-09/data_2018-09-03_to_2018-09-09_train_shuffled"
valid_double_multi = "/home/zhoutong/data/apus_ad/data_2018-09-03_to_2018-09-09/data_2018-09-03_to_2018-09-09_valid_shuffled"
data_g_double_multi = DataGenerator(train_path=train_double_multi, valid_path=valid_double_multi, max_numeric=[999, 10001, 13])
train_generator_d = data_g_double_multi.get_apus_ad_train_generator()

##########
# 数据准备：LR使用的特征（一个多值离散）
train_single_multi = "/home/zhoutong/data/apus_ad/data_2018-09-21_to_2018-09-26/2018-09-21_to_2018-09-25_shuffled"
valid_single_multi = "/home/zhoutong/data/apus_ad/data_2018-09-21_to_2018-09-26/2018-09-26_to_2018-09-26_shuffled"
data_g_single_multi = DataGenerator(train_path=train_single_multi, valid_path=valid_single_multi, max_numeric=[2])
train_generator_s = data_g_single_multi.get_apus_ad_train_generator()

batch_info_d = list(itertools.islice(train_generator_d,0, 10))
batch_info_s = list(itertools.islice(train_generator_s,0, 10))

##########
# 解析函数
def get_sparse_idx(input_df):
            result = []
            for i in range(len(input_df)):
                for j in input_df[i]:
                    result.append([i, j])
            return np.array(result)
def get_sparse_tensor_from(input_df, inp_tensor_shape):
    """
    这里构造的稀疏向量,其indices是自增数,没有实际用途
    其values是input_df内部的值
        - 例如使用 Xi_multi_hot,则生成的稀疏向量values为特征在featureMap中的索引
        - 使用 Xv_multi_hot,则生成的稀疏向量values为特征的"权重",基本都为 1.0
    :param input_df:
    :param inp_tensor_shape:
    :return:
    """
    tensor_values_ = []
    tensor_indices_ = []
    for idx in range(len(input_df)):
        tensor_values_.extend(input_df[idx])
        tensor_indices_.extend([[idx,v] for v in range(len(input_df[idx]))])
    sp_tensor = tf.SparseTensorValue(indices=tensor_indices_, values=tensor_values_, dense_shape=inp_tensor_shape)
    return sp_tensor
def get_payload_gpu(gpu_cnt, payload,*allData_inp):
    result = []
    start = gpu_cnt*payload
    end = (gpu_cnt+1)*payload
    start
    end
    for data in allData_inp:
        result.append(data[start:end])
    return result

## apus_ad | 使用DataSet输入数据

In [ ]:

batch_info_zipped = [np.array(x) for x in zip(*batch_info_s)]

y_inp = batch_info_zipped[0]
payload_per_gpu=len(y_inp)

i=0
# allData=[y, Xi_numeric, Xv_numeric, Xi_category, Xv_category, Xi_multi_hot_list, Xv_multi_hot_list]
allData = get_payload_gpu(i, payload_per_gpu, *batch_info_zipped)
[y, Xi_numeric, Xv_numeric, Xi_category, Xv_category, Xi_multi_hot_list, Xv_multi_hot_list] = allData

tensor_dense_shape=[len(y_inp),18840]
# 整体特征汇总
Xi_total = []
Xv_total = []
for row_idx in range(len(Xi_numeric)):
    Xi_total.append(list(Xi_numeric[row_idx])+list(Xi_category[row_idx])+reduce(lambda a,b: a+b, list(Xi_multi_hot_list[row_idx])))
    Xv_total.append(list(Xv_numeric[row_idx])+list(Xv_category[row_idx])+reduce(lambda a,b: a+b, list(Xv_multi_hot_list[row_idx])))
total_idx_sp = get_sparse_tensor_from(Xi_total,tensor_dense_shape)
total_value_sp = get_sparse_tensor_from(Xv_total,tensor_dense_shape)

print("feat_idx:\n")
total_idx_sp.indices[:5]
total_idx_sp.values[:5]
print("feat_value:\n")
total_value_sp.indices[:5]
total_value_sp.values[:5]

# 连续特征
v_numeric_sparse = np.reshape(Xv_numeric, -1)
v_category_sparse = np.reshape(Xv_category, -1)
# 离散特征
idx_numeric_sparse = get_sparse_idx(Xi_numeric)
idx_category_sparse = get_sparse_idx(Xi_category)

print("category_sparse:")
idx_category_sparse
v_category_sparse


# 多值离散特征 第0个 作示例
Xi_multi_hot = [idx_list[0] for idx_list in Xi_multi_hot_list]
# Xi_multi_hot_tensor = get_sparse_tensor_from(Xi_multi_hot, inp_tensor_shape=tensor_dense_shape)

Xv_multi_hot = [value_list[0] for value_list in Xv_multi_hot_list]
Xv_multi_hot_tensor = get_sparse_tensor_from(Xv_multi_hot, inp_tensor_shape=tensor_dense_shape)
# Xv_multi_hot_tensor

# Xi_numeric,Xv_numeric = allData[1:3]

# idx_numeric_sparse = get_sparse_idx(Xi_numeric)
# v_numeric_sparse = np.reshape(Xv_numeric,-1)
# tensor_dense_shape = [1024, 18850]
# input_sp_tensor = tf.SparseTensorValue(indices=idx_numeric_sparse, 
#                                        values=v_numeric_sparse, 
#                                        dense_shape=tensor_dense_shape)

################
# TensorFlow部分
# sp_tensor = tf.sparse_placeholder(tf.float32)
# dataset = tf.data.Dataset.from_tensor_slices(sp_tensor).batch(tensor_dense_shape[0])
# # dataset = tf.data.Dataset.from_tensor_slices(sp_tensor).batch(10)
# iterator = dataset.make_initializable_iterator()
# next_el = iterator.get_next()
# feed_dict = {sp_tensor: input_sp_tensor}
# with tf.Session() as sess:
#     sess.run(sp_tensor, feed_dict)
#     sess.run(iterator.initializer, feed_dict)
#     next_el

# dataset_idx_of_numeric = tf.data.Dataset.from_tensors(idx_numeric_sparse)
# dataset_value_of_numeric = tf.data.Dataset.from_tensors(tf.cast(v_numeric_sparse,'float'))
# dataset_idx_of_numeric.output_types
# dataset_idx_of_numeric.output_shapes

# 使用TFRecord+DataSet作为输入
- ## apus_ad | DeepFM | config
- ## apus_ad | DeepFM | weights
- ## apus_ad | DeepFM | model
- ## apus_ad | DeepFM | input

## apus_ad | DeepFM | config

In [93]:
#***** 存跃的数据（midas） *****
# deprecated
class config_midas():
    basePath = "/home/zhoutong/data/apus_ad/midas/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11_intersectLR"
#     basePath = "/home/zhoutong/data/apus_ad/midas/tfrecord_2018-10-10_to_2018-10-17_and_2018-10-18_to_2018-10-21_intersectLR"
    train_tfrecord_file = basePath+"/train.tfrecord.gz"
    valid_tfrecord_file = basePath+"/valid.tfrecord.gz"
    info_file = basePath+"/info.json"
    
    with open(info_file,"r+") as f:
        info = "".join(f.readlines())
        result = json.loads(info)
        
    # fields
    global_all_fields = [
    "label", "ad_info__ad_type", "ad_info__app_package_name",
    "ad_info__budget_unit", "user_behavior__ad_position_id_s",
    "user_behavior__app_version_s", "user_behavior__client_ip_s",
    "user_behavior__country_s", "user_behavior__hour_s",
    "user_behavior__language_s", "user_behavior__package_name_s",
    "user_profile_app__install_app_list"]
    
    global_numeric_fields = ['ad_info__budget_unit']
    global_multi_hot_fields = ['user_profile_app__install_app_list']
    global_one_hot_fields = []
    for i in global_all_fields:
        if i not in global_numeric_fields and i not in global_multi_hot_fields and i != "label":
            global_one_hot_fields.append(i)
    # feature_size
    feature_size = 1451685+1
    # field_size
    multi_hot_field_size = len(global_multi_hot_fields)
    numeric_field_size = len(global_numeric_fields)
    one_hot_field_size = len(global_one_hot_fields)
    
    # 连续特征的索引号要单独给出来，方便后续构造idx_sparse_tensor
    tmp_map_num_f = {'ad_info__budget_unit':1291744}
    max_numeric = {"ad_info__budget_unit": 2.0}
    # shape
    global_dense_shape = [1024,feature_size]
    
    # deepfm_param
    embedding_size = 8
    dropout_fm = [1.0, 1.0]
    deep_layers = [128, 64, 32, 16]
    dropout_deep = [1.0, 0.9, 0.9, 0.9, 0.9]
    deep_layers_activation = tf.nn.relu
    epoch=10
    batch_size= 1024*3
    learning_rate= 0.001
    optimizer_type="adam"
    batch_norm= 1
    batch_norm_decay= 0.99
    l2_reg=-0.01
    verbose= True
    eval_metric=roc_auc_score
    random_seed=2017
    gpu_num=1
    is_debug=False
    
    summary_save_dir = "/home/zhoutong/midas_summary"
    model_save_dir = "/home/zhoutong/midas_model"
    
    def get_now():
        return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(time.time()))

class config_midas_():
    basePath = "/home/zhoutong/data/apus_ad/midas/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11_filterRepeatView"
#     basePath = "/home/zhoutong/data/apus_ad/midas/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11_intersectLR"
    train_tfrecord_file = basePath+"/train.tfrecord.gz"
    valid_tfrecord_file = basePath+"/valid.tfrecord.gz"
    info_file = basePath+"/info.json"
    
    # fields
    with open(info_file,"r+") as f:
        info = "".join(f.readlines())
        result = json.loads(info)

    fieldInfo = result['allField']
    global_all_fields = fieldInfo['all_fields'].split(",")
    global_numeric_fields = fieldInfo['numeric_fields'].split(",")
    global_multi_hot_fields = fieldInfo['multi_hot_fields'].split(",")
    global_one_hot_fields = []
    for i in global_all_fields:
        if i not in global_numeric_fields and i not in global_multi_hot_fields and i != "label":
            global_one_hot_fields.append(i)
    
    statisticInfo = result['statistic']
    # feature_size
    feature_size = statisticInfo['feature_size']+1
    # field_size
    multi_hot_field_size = len(global_multi_hot_fields)
    numeric_field_size = len(global_numeric_fields)
    one_hot_field_size = len(global_one_hot_fields)
    
    # 连续特征的索引号要单独给出来，方便后续构造idx_sparse_tensor
    tmp_map_num_f = result['numericFieldMap']#{'ad_info__budget_unit':1291744}
    max_numeric = result['numericMax']#{"ad_info__budget_unit": 2.0}
    
    # deepfm_param
    embedding_size = 1
    dropout_fm = [1.0, 1.0]
    deep_layers = [2,1]
    dropout_deep = [1.0, 0.9, 0.9, 0.9, 0.8]
    deep_layers_activation = tf.nn.relu
    epoch=10
    batch_size= 1024*3
    learning_rate= 0.001
    optimizer_type="adam"
    batch_norm= 1
    batch_norm_decay= 0.9
    l2_reg=-0.0001
    verbose= True
    eval_metric=roc_auc_score
    random_seed=2017
    gpu_num=1
    is_debug=False
    
    # shape
    global_dense_shape = [batch_size,feature_size]
    
    summary_save_dir = "/home/zhoutong/midas_summary"
    model_save_dir = "/home/zhoutong/midas_model"
    
    def get_now():
        return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(time.time()))

In [102]:
#***** 国盛的数据（starksdk） *****
class config_starksdk():
    basePath = "/home/zhoutong/data/apus_ad/starksdk/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11"
    train_tfrecord_file = basePath+"/train.tfrecord.gz"
    valid_tfrecord_file = basePath+"/valid.tfrecord.gz"
    info_file = basePath+"/info.json"
    
    # fields
    global_all_fields = ["label","user_profile_app__install_app_list","ad_info__advertiser","ad_info__advertiser_gp_class",
    "user_profile_basic__dpi","user_profile_basic__network_type","user_profile_basic__locale",
    "user_profile_basic__manufacturer","user_profile_basic__model","user_profile_basic__country"]
    global_numeric_fields = []
    global_multi_hot_fields = ['user_profile_app__install_app_list']
    global_one_hot_fields = []
    for i in global_all_fields:
        if i not in global_numeric_fields and i not in global_multi_hot_fields and i != "label":
            global_one_hot_fields.append(i)
    feature_size = 1087368+1
    # field_size ( 如果没有该类型的特征，后续会默认填充一个为0的tensor，这里需要把size置为1)
    multi_hot_field_size = 1 if len(global_multi_hot_fields)==0 else len(global_multi_hot_fields)
    numeric_field_size = 1 if len(global_numeric_fields)==0 else len(global_numeric_fields)
    one_hot_field_size = 1 if len(global_one_hot_fields)==0 else len(global_one_hot_fields)
    
    # 连续特征的索引号要单独给出来，方便后续构造idx_sparse_tensor
    tmp_map_num_f ={}# {'ad_info__budget_unit':0}
    max_numeric = {}#{"ad_info__budget_unit": 2.0}
    # shape
    global_dense_shape = [1024,feature_size]
    
    # deepfm_param
    embedding_size = 8
    dropout_fm = [1.0, 1.0]
    deep_layers = [128, 64, 32, 16]
    dropout_deep = [1.0, 0.9, 0.9, 0.9, 0.9]
    deep_layers_activation = tf.nn.relu
    epoch=10
    batch_size= 1024*6
    learning_rate= 0.001
    optimizer_type="adam"
    batch_norm= 1
    batch_norm_decay= 0.99
    l2_reg=-0.00001
    verbose= True
    eval_metric=roc_auc_score
    random_seed=2017
    gpu_num=1
    is_debug=False
    
    summary_save_dir = "/home/zhoutong/starksdk_summary"
    model_save_dir = "/home/zhoutong/starksdk_model"
    
    def get_now():
        return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(time.time()))

In [103]:
def get_dict(instance):
    dict_ = instance.__dict__
    keys = [i for i in dict_.keys() if "__" not in i and i != "get_dict" ]   
    return {key:dict_[key] for key in keys}

config = config_starksdk
# config = config_midas_
# config = config_midas_noLR

print("json's info:")
import json
with open(config.info_file,"r+") as f:
    info = f.read()
    result = json.loads(info)
for key,value in result.items():
    print(key,"--")
    for key_,value_ in value.items():
        print("    ",key_,"=",value_)
print("batch_cnt:",(result['statistic']['train_pos']+result['statistic']['train_neg'])/config.batch_size)

json's info:
statistic --
     train_pos = 514076
     valid_neg = 110887
     train_neg = 2051521
     multi_hot_f_size = 1
     field_size = 10
     valid_pos = 193486
     feature_size = 1087368
     numeric_f_size = 0
numericFieldMap --
allField --
     all_fields = {'empty': False, 'traversableAgain': True}
     numeric_fields = {'empty': True, 'traversableAgain': True}
     multi_hot_fields = {'empty': False, 'traversableAgain': True}
featureCnt --
     user_profile_basic__manufacturer = 3199
     ad_info__advertiser = 72980
     ad_info__advertiser_gp_class = 35
     user_profile_basic__network_type = 9
     user_profile_basic__model = 17118
     user_profile_basic__locale = 514
     user_profile_basic__country = 234
     user_profile_basic__dpi = 150
     user_profile_app__install_app_list = 993130
batch_cnt: 417.57763671875


## apus_ad | DeepFM | weights

In [96]:
def _initialize_weights():
        multi_hot_field_size = config.multi_hot_field_size
        one_hot_field_size = config.one_hot_field_size
        numeric_field_size = config.numeric_field_size
        feature_size = config.feature_size
        embedding_size = config.embedding_size
        deep_layers = config.deep_layers
        
        weights = dict()
        # embeddings
        weights["feature_embeddings"] = tf.Variable(
            tf.random_normal([feature_size, embedding_size], -0.01, 0.01),
            name="feature_embeddings")  # feature_size * K
        # FM first-order weights
        weights["feature_bias"] = tf.Variable(
            tf.random_uniform([feature_size, 1], -0.01, 0.01), name="feature_bias")  # feature_size * 1
        # deep layers
        # 总输入元个数为 : (涉及emb的特征个数) * embedding_size + 连续特征个数
        input_size_emb = (multi_hot_field_size+one_hot_field_size) * embedding_size + numeric_field_size
        glorot = np.sqrt(2.0 / (input_size_emb + deep_layers[0]))
        weights["layer_0"] = tf.Variable(
            initial_value=np.random.normal(loc=0, scale=glorot, size=(input_size_emb, deep_layers[0])), 
            dtype=np.float32,
            name="w_layer_0")
        weights["bias_0"] = tf.Variable(np.random.normal(loc=0, scale=glorot, size=(1, deep_layers[0])),
                                        dtype=np.float32, name="b_layer_0")  # 1 * layers[0]
        for i in range(1, len(deep_layers)):
            glorot = np.sqrt(2.0 / (deep_layers[i - 1] + deep_layers[i]))
            weights["layer_%d" % i] = tf.Variable(
                np.random.normal(loc=0, scale=glorot, size=(deep_layers[i - 1], deep_layers[i])),
                dtype=np.float32, name="w_layer_%d" % i)  # layers[i-1] * layers[i]
            weights["bias_%d" % i] = tf.Variable(
                np.random.normal(loc=0, scale=glorot, size=(1, deep_layers[i])),
                dtype=np.float32, name="b_layer_%d" % i)  # 1 * layer[i]
        # final concat projection layer
        ################
        # fm的y_first_order已经被提前求和了，所以只需要给它一个权重
        # （因为在weights["feature_bias"]中已经有部分作为“权重”乘上了y_first_order的特征值，然后求和，相当于每个一阶特征都有自己的隐向量x权重(来自w["feature_bias"])
        ################
        cocnat_input_size_emb = 1 + embedding_size + deep_layers[-1]
        glorot = np.sqrt(2.0 / (cocnat_input_size_emb + 1))
        weights["concat_projection"] = tf.Variable(
            np.random.normal(loc=0, scale=glorot, size=(cocnat_input_size_emb, 1)),
            dtype=np.float32, name="concat_projection")  # layers[i-1]*layers[i]
        weights["concat_bias"] = tf.Variable(tf.constant(0.01), dtype=np.float32, name="concat_bias")
        return weights

## apus_ad | DeepFM | model

In [97]:
# params for deepfm
def run_deepfm(self,weights,inp_list,train_phase,dropout_keep_fm,dropout_keep_deep):
    with tf.name_scope("gen_feat_total"):
        feat_total_idx_sp,feat_total_value_sp = _get_total_feature(inp_list)
    with tf.name_scope("gen_feat_multi_hot"):
        feat_multi_hot_idx_sp_list = _get_multi_hot_idx_list(inp_list)
        feat_multi_hot_value_sp_list = _make_multi_hot_value_list(feat_multi_hot_idx_sp_list)
    with tf.name_scope("gen_feat_numeric"):
        feat_numeric_sp = _get_numeric_sp(inp_list,config.batch_size)
    with tf.name_scope("gen_feat_category"):
        feat_category_sp = _get_category_sp(inp_list)
    return _deep_fm_graph(self,weights,dropout_keep_fm,dropout_keep_deep,feat_total_idx_sp, feat_total_value_sp,
                      feat_multi_hot_idx_sp_list, feat_multi_hot_value_sp_list,
                      feat_numeric_sp, feat_category_sp, train_phase)

def _deep_fm_graph(self,weights, dropout_keep_fm,dropout_keep_deep,feat_total_idx_sp, feat_total_value_sp,
                      feat_multi_hot_idx_sp_list, feat_multi_hot_value_sp_list,
                      feat_numeric_sp, feat_category_sp, train_phase):
        def batch_norm_layer(x, inp_train_phase, scope_bn,inp_batch_norm_decay):
            bn_train = batch_norm(x, decay=inp_batch_norm_decay, center=True, scale=True, updates_collections=None,
                                  is_training=True, reuse=None, trainable=True, scope=scope_bn)
            bn_inference = batch_norm(x, decay=inp_batch_norm_decay, center=True, scale=True, updates_collections=None,
                                      is_training=False, reuse=True, trainable=True, scope=scope_bn)
            z = tf.cond(inp_train_phase, lambda: bn_train, lambda: bn_inference)
            return z
        
#         dropout_keep_fm = self.dropout_fm
#         dropout_keep_deep = self.dropout_deep
        numeric_feature_size = self.numeric_field_size
        onehot_field_size = self.one_hot_field_size
        multi_hot_field_size = self.multi_hot_field_size
        embedding_size = self.embedding_size
        deep_layers_activation = self.deep_layers_activation
        batch_norm_decay = self.batch_norm_decay
        
        deep_input_size = multi_hot_field_size + onehot_field_size
        
        # ---------- FM component ---------
        with tf.name_scope("FM"):
            # ---------- first order term ----------
            with tf.name_scope("1st_order"):
                y_first_order = tf.nn.embedding_lookup_sparse(
                    weights["feature_bias"],
                    sp_ids=feat_total_idx_sp,
                    sp_weights=feat_total_value_sp,
                    combiner="sum")
                y_first_order = tf.nn.dropout(
                    y_first_order,
                    dropout_keep_fm[0],
                    name="y_first_order_dropout")
            # ---------- second order term ---------------
            with tf.name_scope("2nd_order"):
                # sum_square part
                summed_features_emb_square = tf.square(
                    tf.nn.embedding_lookup_sparse(
                        weights["feature_embeddings"],
                        sp_ids=feat_total_idx_sp,
                        sp_weights=feat_total_value_sp,
                        combiner="sum"))
                # square_sum part
                squared_sum_features_emb = tf.nn.embedding_lookup_sparse(
                    tf.square(weights["feature_embeddings"]),
                    sp_ids=feat_total_idx_sp,
                    sp_weights=tf.square(feat_total_value_sp),
                    combiner="sum")
                # second order
                y_second_order = 0.5 * tf.subtract(
                    summed_features_emb_square,
                    squared_sum_features_emb)  # None * K
                y_second_order = tf.nn.dropout(y_second_order,
                                               dropout_keep_fm[1])  # None * K
        # ---------- Deep component -------
        with tf.name_scope("Deep"):
            # total_embedding 均值 用户的multi-hot one-hot特征都取到embedding作为DNN输入
            with tf.name_scope("total_emb"):
                # feat_one_hot = tf.sparse_add(feat_numeric_sp, feat_category_sp)
                feat_one_hot = feat_category_sp
                one_hot_embeddings = tf.nn.embedding_lookup(
                    weights["feature_embeddings"], feat_one_hot.indices[:, 1])
                one_hot_embeddings = tf.reshape(
                    one_hot_embeddings,
                    shape=(-1, onehot_field_size, embedding_size))
                multi_hot_embeddings = []
                for feat_idx_sp, feat_value_sp in zip(
                        feat_multi_hot_idx_sp_list, feat_multi_hot_value_sp_list):
                    emb = tf.nn.embedding_lookup_sparse(
                        weights["feature_embeddings"],
                        sp_ids=feat_idx_sp,
                        sp_weights=feat_value_sp,
                        combiner="mean")
                    emb = tf.reshape(emb, shape=[-1, 1, embedding_size])
                    multi_hot_embeddings.append(emb)
                total_embeddings = tf.concat(
                    values=[one_hot_embeddings] + multi_hot_embeddings, axis=1)
            # input
            with tf.name_scope("input"):
                # 把连续特征不经过embedding直接输入到NN
                feat_numeric_sp_dense = tf.cast(
                    tf.reshape(
                        feat_numeric_sp.values, shape=(-1, numeric_feature_size)),
                    tf.float32)
                y_deep_input = tf.reshape(
                    total_embeddings,
                    shape=[-1, deep_input_size * embedding_size])  # None * (F*K)
                y_deep_input = tf.concat([y_deep_input, feat_numeric_sp_dense],
                                         axis=1)
                y_deep_input = tf.nn.dropout(y_deep_input, dropout_keep_deep[0])
            # layer0
            with tf.name_scope("layer0"):
                y_deep_layer_0 = tf.add(
                    tf.matmul(y_deep_input, weights["layer_0"]), weights["bias_0"])
                y_deep_layer_0 = batch_norm_layer(
                    y_deep_layer_0, inp_train_phase=train_phase, scope_bn="bn_0",inp_batch_norm_decay=batch_norm_decay)
                y_deep_layer_0 = deep_layers_activation(y_deep_layer_0)
                y_deep_layer_0 = tf.nn.dropout(y_deep_layer_0, dropout_keep_deep[1])
            # layer1
            with tf.name_scope("layer1"):
                y_deep_layer_1 = tf.add(
                    tf.matmul(y_deep_layer_0, weights["layer_1"]),
                    weights["bias_1"])
                y_deep_layer_1 = batch_norm_layer(
                    y_deep_layer_1, inp_train_phase=train_phase, scope_bn="bn_1",inp_batch_norm_decay=batch_norm_decay)
                y_deep_layer_1 = deep_layers_activation(y_deep_layer_1)
                y_deep_layer_1 = tf.nn.dropout(y_deep_layer_1, dropout_keep_deep[2])
#             # layer2
#             with tf.name_scope("layer2"):
#                 y_deep_layer_2 = tf.add(
#                     tf.matmul(y_deep_layer_1, weights["layer_2"]),
#                     weights["bias_2"])
#                 y_deep_layer_2 = batch_norm_layer(
#                     y_deep_layer_2, inp_train_phase=train_phase, scope_bn="bn_2",inp_batch_norm_decay=batch_norm_decay)
#                 y_deep_layer_2 = deep_layers_activation(y_deep_layer_2)
#                 y_deep_layer_2 = tf.nn.dropout(y_deep_layer_2, dropout_keep_deep[3])
            # layer3
#             with tf.name_scope("layer3"):
#                 y_deep_layer_3 = tf.add(
#                     tf.matmul(y_deep_layer_0, weights["layer_3"]),
#                     weights["bias_3"])
#                 y_deep_layer_3 = batch_norm_layer(
#                     y_deep_layer_3, inp_train_phase=train_phase, scope_bn="bn_3",inp_batch_norm_decay=batch_norm_decay)
#                 y_deep_layer_3 = deep_layers_activation(y_deep_layer_3)
#                 y_deep_layer_3 = tf.nn.dropout(y_deep_layer_3, dropout_keep_deep[4])
        # ---------- DeepFM ---------------
        with tf.name_scope("DeepFM"):
            y_deep_out = y_deep_layer_1
#             y_deep_out = y_deep_layer_3
            concat_input = tf.concat([y_first_order, y_second_order, y_deep_out], axis=1)
#             concat_input = tf.concat([y_first_order, y_second_order], axis=1)
            out = tf.add(
                tf.matmul(concat_input, weights["concat_projection"]),
                weights["concat_bias"])
    
        return tf.nn.sigmoid(out)

## apus_ad | DeepFM | input

In [98]:
# 解析 TF Example 文件
def _decode(serialized_example):
    feature_structure = {}
    for field in config.global_all_fields:
        if field == "label":
            feature_structure[field]=tf.FixedLenFeature([], dtype=tf.int64)
        elif field in config.global_multi_hot_fields:
            feature_structure[field] = tf.VarLenFeature(dtype=tf.int64)
        elif field in config.global_numeric_fields:
            feature_structure[field] = tf.FixedLenFeature([],dtype=tf.float32)
        else:
            feature_structure[field]=tf.FixedLenFeature([], dtype=tf.int64)
    parsed_features = tf.parse_single_example(serialized_example, feature_structure)
    return parsed_features

# 目前label是string要转成int64，后续会改
def _convert_label_toInt64(parsed_features):
    label = parsed_features['label']
    parsed_features['label'] = tf.string_to_number(label,out_type=tf.int64)
    return parsed_features

# 连续特征归一化
def _normalize(parsed_features):
    for num_f in config.global_numeric_fields:
        max_v = config.max_numeric[num_f] 
        parsed_features[num_f] = parsed_features[num_f] / max_v - 0.5
    return parsed_features

# 把连续特征的idx加进去，跟样本一起出现batch_size次
def _add_idx_of_numeric(parsed_features):
    for field in config.global_numeric_fields:
        parsed_features[field+"_idx"] = tf.cast(config.tmp_map_num_f[field],tf.int64)
    return parsed_features

def __add_idx_to_tensor(inp_tensor):
    idx = tf.range(tf.shape(inp_tensor)[0])
    idx_2d = tf.reshape(idx,[-1,1])
    idx_2d_full = tf.cast(tf.tile(idx_2d,[1,tf.shape(inp_tensor)[1]]),dtype=inp_tensor.dtype)
    result = tf.concat([tf.reshape(idx_2d_full,[-1,1]),tf.reshape(inp_tensor,[-1,1])],axis=1)
    return result

def _get_numeric_sp(inp_dict,batch_size):
    if len(config.global_numeric_fields) !=0:
        idx_to_stack=[]
        value_to_stack=[]
        for field in config.global_numeric_fields:
            idx_to_stack.append(inp_dict[field+"_idx"])
            value_to_stack.append(inp_dict[field])
        idx_dense = __add_idx_to_tensor(tf.transpose(tf.stack(idx_to_stack)))
        value_dense = tf.reshape(tf.transpose(tf.stack(value_to_stack)),[-1])
    else:
        # 为了保持连贯性，没有连续特征会构造“一个”连续特征，全为0
        idx_dense = tf.constant([[i,0] for i in range(batch_size)],dtype=tf.int64)
        value_dense = tf.constant([0.0]*batch_size,dtype=tf.float32)
    return tf.SparseTensor(indices=idx_dense, values=value_dense, dense_shape=[batch_size,config.numeric_field_size])

def _get_category_sp(inp_dict):
    if len(config.global_one_hot_fields) != 0:
        idx_to_stack=[]
        value_to_stack=[]
        for field in config.global_one_hot_fields:
            idx_to_stack.append(inp_dict[field])
            value_to_stack.append(tf.ones_like(inp_dict[field],dtype=tf.float32))
            idx_dense = __add_idx_to_tensor(tf.transpose(tf.stack(idx_to_stack)))
            value_dense = tf.reshape(tf.transpose(tf.stack(value_to_stack)),[-1])
    else:
        idx_dense = tf.constant([[0,0]],dtype=tf.int64)
        value_dense = tf.constant([0.0],dtype=tf.float32)
    return tf.SparseTensor(indices=idx_dense, values=value_dense, dense_shape=config.global_dense_shape)

# 这个multi_hot的默认值没有做完，和numeric之前一样的错误，没有构造batch个稀疏特征
def _get_multi_hot_idx_list(inp_dict):
    multi_hot_idx_list = []
    if len(config.global_multi_hot_fields) != 0:
        for field in config.global_multi_hot_fields:
            multi_hot_idx_list.append(inp_dict[field])
    else:
        multi_hot_idx_list.append(tf.SparseTensor(indices=[[0,0]], values=[0.0], dense_shape=config.global_dense_shape))
    return multi_hot_idx_list

def _make_multi_hot_value_list(feat_idx_list):
    multi_hot_value_list = []
    if len(feat_idx_list) !=0:
        multi_hot_value_list=[tf.SparseTensor(indices=sparse.indices,values=tf.ones_like(sparse.values,dtype=tf.float32),dense_shape=sparse.dense_shape) for sparse in feat_idx_list]
    else:
        multi_hot_value_list.append(tf.SparseTensor(indices=[[0,0]], values=[0.0], dense_shape=config.global_dense_shape))
    return multi_hot_value_list

def _get_total_feature(inp_dict):
    idx_to_stack = []
    value_to_stack = []
    # sparse_tensor来表示multi_hot
    multi_hot_idx_sparse_list = []
    for field in config.global_all_fields:
        if field in config.global_multi_hot_fields:
            multi_hot_idx_sparse_list.append(inp_dict[field])
        if field in config.global_numeric_fields:
            idx_to_stack.append(inp_dict[field+"_idx"])
            value_to_stack.append(inp_dict[field])
            pass
        if field in config.global_one_hot_fields:
            idx_to_stack.append(inp_dict[field])
            value_to_stack.append(tf.ones_like(inp_dict[field],dtype=tf.float32))
            pass
    # sparse_tensor的values中原来都是特征索引，替换成1.0
    multi_hot_value_sparse_list = [tf.SparseTensor(indices=sparse.indices, values=tf.ones_like(sparse.values,dtype=tf.float32), dense_shape=sparse.dense_shape) for sparse in multi_hot_idx_sparse_list]
    # idx sparse of numeric+onehot 
    idx_dense = tf.transpose(tf.stack(idx_to_stack))
    idx_sparse = tf.contrib.layers.dense_to_sparse(tensor=idx_dense,eos_token=-1)
    # value sparse of numeric+onehot 
    value_dense = tf.transpose(tf.stack(value_to_stack))
    value_sparse = tf.contrib.layers.dense_to_sparse(tensor=value_dense,eos_token=-1)
    
    total_idx_sparse = tf.sparse_concat(axis=1,sp_inputs=[idx_sparse]+ multi_hot_idx_sparse_list)
    total_value_sparse = tf.sparse_concat(axis=1,sp_inputs=[value_sparse] + multi_hot_value_sparse_list)
    return total_idx_sparse, total_value_sparse
    
def get_input(tfrecord_path):
    with tf.name_scope("dataset"):
        dataset = tf.data.TFRecordDataset(tfrecord_path,compression_type = compression_type)
        dataset = (dataset.map(_decode)
                   .map(_normalize)
                   .map(_add_idx_of_numeric))
        dataset = (dataset.shuffle(5*config.batch_size)
                   .batch(config.batch_size,drop_remainder=True))
        iterator = dataset.make_initializable_iterator()
    return iterator


## apus_ad | DeepFM | Graph

In [99]:
# 构造图
# train_tfrecord_file = "/home/zhoutong/data/apus_ad/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11/train.tfrecord.gz"
# valid_tfrecord_file = "/home/zhoutong/data/apus_ad/tfrecord_2018-09-21_to_2018-10-04_and_2018-10-05_to_2018-10-11/valid.tfrecord.gz"

compression_type = "GZIP"
tf.set_random_seed(2018)
graph = tf.Graph()

with graph.as_default():
    weights = _initialize_weights()
    total_parameters = 0
    for variable in weights.values():
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print("total_parameters cnt : %s" % total_parameters)
        
    inp_tfrecord_path = tf.placeholder(dtype=tf.string, name="tfrecord_path")
    # optimizer
    _optimizer = tf.train.AdamOptimizer(learning_rate=config.learning_rate, beta1=0.9, beta2=0.999,epsilon=1e-8)

    # prepare
    # inp_next_dict     key: decode时使用的字符串，value: tensor
    # placeholder_dict  key: decode时使用的字符串，value: placeholder
    #                   目的：为了让后面的流程都使用placeholder进行,这样存储模型可以以这些placeholder为输入口
    # ori_feed_dict     key: placeholder        value: tensor
    #                   目的：直接sess.run(ori_feed_dict)就可以得到后续流程需要的placeholder的feed_dict;
    inp_iterator = get_input(inp_tfrecord_path)
    inp_next_dict = inp_iterator.get_next()

    # 构造placeholder输入，方便模型文件restore后的使用
    # 这里实际上只是把 inp_next 这个“源字典”的 value 都用placeholder替换了，key未变
    placeholder_dict = {}
    with tf.name_scope("input"):
        # train_phase放到这里只是为了共享同一个name_scope
        train_phase = tf.placeholder(dtype=tf.bool,name="train_phase")
        placeholder_dict["train_phase"]=train_phase
        for k,v in inp_next_dict.items():
            if k in config.global_multi_hot_fields:
                placeholder_dict[k]=tf.sparse_placeholder(dtype=tf.int64,name=k)
            elif k in config.global_numeric_fields:
                placeholder_dict[k]=tf.placeholder(dtype=tf.float32,name=k)
            else:
                placeholder_dict[k]=tf.placeholder(dtype=tf.int64,name=k)
        # 构造一个feed_dict在训练的时候自动就用它，取placeholder为key，取“源字典”的value为value
        ori_feed_dict = {placeholder_dict[k] : inp_next_dict[k] for k,v in inp_next_dict.items()}
    
    # deepfm
    dropout_keep_fm = tf.cond(train_phase,lambda:config.dropout_fm,lambda:[1.0]*len(config.dropout_fm))
    dropout_keep_deep = tf.cond(train_phase,lambda:config.dropout_deep,lambda:[1.0]*len(config.dropout_deep))
    
    deepfm_output = run_deepfm(config,weights,placeholder_dict,train_phase,dropout_keep_fm,dropout_keep_deep)
    with tf.name_scope("output"):
        pred = tf.reshape(deepfm_output,[-1],name="pred")
    # label
    label_op = placeholder_dict['label']
    
    label_cnt_op = tf.bincount(tf.cast(label_op,tf.int32))
    # loss
    empirical_risk = tf.reduce_mean(tf.losses.log_loss(label_op, pred))
    loss_op = empirical_risk
    if config.l2_reg>0:
        structural_risk = tf.contrib.layers.l2_regularizer(config.l2_reg)(weights["concat_projection"])
        for i in range(len(config.deep_layers)):
            structural_risk += tf.contrib.layers.l2_regularizer(config.l2_reg)(weights["layer_%d"%i])
        tf.summary.scalar('structural_risk_L2',structural_risk)
        loss_op = empirical_risk + structural_risk
    
    # optimize
    grad = _optimizer.compute_gradients(loss_op)
    optimize_op = _optimizer.apply_gradients(grad)
    
    # summary (tensorboard)
    tf.summary.scalar('total_loss', loss_op)
    tf.summary.scalar('empirical_risk_logloss',empirical_risk)

    for g,v in grad:
        if g is not None:
            _=tf.summary.histogram(v.op.name+"/gradients",g)
    for v in tf.trainable_variables():
        _=tf.summary.histogram(v.name.replace(":0","/value"),v)
    merge_summary = tf.summary.merge_all()#调用sess.run运行图，生成一步的训练过程数据, 是一个option
    writer = tf.summary.FileWriter(config.summary_save_dir+"/summary_%s" % config.get_now(), graph)
    init_op = tf.global_variables_initializer()

    inputs = placeholder_dict
    outputs = {"pred":pred}
    mySaver = tf.train.Saver(max_to_keep=2)
    
train_feed={train_phase:True,inp_tfrecord_path:config.train_tfrecord_file}
valid_feed={train_phase:False,inp_tfrecord_path:config.valid_tfrecord_file}


total_parameters cnt : 3104325


/home/zhoutong/python3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


<tf.Tensor 'total_loss:0' shape=() dtype=string>

<tf.Tensor 'empirical_risk_logloss:0' shape=() dtype=string>

## apus_ad | DeepFM | session

In [100]:
import logging
import time
import datetime
logger = logging.getLogger()
def setup_file_logger(log_file):
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)

def log(message):
    new_m = "|{}| {}".format(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),message)
    print(new_m)
    logger.info(new_m)



In [101]:
train_feed={train_phase:True,inp_tfrecord_path:config.train_tfrecord_file}
valid_feed={train_phase:False,inp_tfrecord_path:config.valid_tfrecord_file}

def _evaluate(sess,valid_dict):
    pred_deque,label_deque,label_cnt_deque=deque(),deque(),deque()
    batch_cnt = 0
    while True:
        try:
            valid_dict.update(sess.run(ori_feed_dict))
            batch_cnt += 1
            t1 = time.time()
            pred_,label_,label_cnt_= sess.run([pred,label_op,label_cnt_op],valid_dict)
            pred_deque.extend(pred_)
            label_deque.extend(label_)
            label_cnt_deque.append(label_cnt_)
        except tf.errors.OutOfRangeError:
            sys.stdout.write("\n")
            sys.stdout.flush()
            break
        delta_t = time.time() - t1
        sys.stdout.write(f"    valid_batch_cnt: [{batch_cnt:0>3d}] [{delta_t:.2f}s/per]\r")
        sys.stdout.flush()

    pred_arr = np.array(pred_deque)
    label_arr = np.array(label_deque)
    label_cnt_arr = np.array(label_cnt_deque)
    pos=label_cnt_arr[:,1].sum()
    neg = label_cnt_arr[:,0].sum()
    now = time.strftime("|%Y-%m-%d %H:%M:%S| ", time.localtime(time.time()))
    print("%s pos/neg: [%s/%s]" % (now,pos,neg))
    auc = roc_auc_score(label_arr,pred_arr)
    loss = log_loss(label_arr,pred_arr,eps=1e-7)
    return loss,auc

def _simple_save(sess,path,inputs,outputs,global_batch_cnt,auc,use_simple_save = False, use_ONNX=False):
    print("save model at %s" % path)
    if use_simple_save:
#         if(os.path.exists(path)):
#             shutil.rmtree(path)
#             print("路径存在，删除原模型..")
        tf.saved_model.simple_save(sess,path+f"/model_of_auc-{auc:.5f}",inputs,outputs)
    else:
        mySaver.save(sess, path+"/model.ckpt", global_step=global_batch_cnt)
    if use_ONNX:
        onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph)
        onnx_inputs = [v.name for k,v in inputs.items()]
        onnx_outputs = [v.name for k,v in outputs.items()]
        model_proto = onnx_graph.make_model("test", onnx_inputs, onnx_outputs)
        with open(path+"/model.onnx","wb") as f:
            f.write(model_proto.SerializeToString())
    pass


out_time=config.get_now()
setup_file_logger("/home/zhoutong/logging/%s" % out_time)

model_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
model_config.gpu_options.allow_growth = True
sess = tf.Session(config=model_config,graph=graph)


with sess as sess:
    sess.run(init_op)
    global_auc,global_batch_cnt,batch_cnt,epoch_cnt =0,0,0,0
    
    for epoch in range(10):
        epoch_cnt += 1
        batch_cnt = 0
        sess.run(inp_iterator.initializer,train_feed)
        t0=time.time()
        while True:
            try:
                batch_cnt += 1
                global_batch_cnt += 1
                train_feed.update(sess.run(ori_feed_dict))
                run_ops=[optimize_op,loss_op,pred,label_op,label_cnt_op,merge_summary]
                run_result = sess.run(run_ops,train_feed)
                _,loss_,pred_,label_,label_cnt_,merge_summary_ = run_result
                writer.add_summary(merge_summary_,global_batch_cnt)
                neg,pos = label_cnt_
                if batch_cnt % 100 == 0:
                    auc = roc_auc_score(label_,pred_)
                    batch_time = int(time.time()-t0)
                    log(f"[e:{epoch_cnt:0>2d}|b:{batch_cnt:0>4d}] pos/neg:[{pos}/{neg}] logloss:[{loss_:.4f}] auc:[{auc:.4f}] [{batch_time}s]")
                    t0=time.time()
                # 存在严重缺陷，这里如果用valid初始化后，从1001batch开始都会从valid里面拿数据了
    #             if batch_cnt % 1000 ==0:
    #                 sess.run(inp_iterator.initializer,valid_dict)
    #                 logloss,auc=_evaluate(sess,valid_feed)
    #                 now = time.strftime("|%Y-%m-%d %H:%M:%S| ", time.localtime(time.time()))
    #                 print(f"{now} [e:{epoch_cnt:0>2d}|b:{batch_cnt:0>4d} valid] logloss:[{logloss:.5f}] auc:[{auc:.5f}]")
            except tf.errors.OutOfRangeError:
                break
        log(f"[e:{epoch_cnt:0>2d}|b:{batch_cnt:0>4d}] epoch-done")
        sess.run(inp_iterator.initializer,valid_feed)
        logloss,auc=_evaluate(sess,valid_feed)

        log(f"[e:{epoch_cnt:0>2d}|b:{batch_cnt:0>4d} valid] valid_logloss:[{logloss:.5f}] valid_auc:[{auc:.5f}]")
        if global_auc<auc:
            global_auc = auc
            log(f"logloss:[{logloss:.5f}] auc:[{auc:.5f}] global_batch_cnt:[{global_batch_cnt:0>4d}] gonna save model ...")
#             _simple_save(sess,config.model_save_dir+f"/model_example",inputs,outputs,global_batch_cnt)
            _simple_save(sess,config.model_save_dir+f"/model_{out_time}",inputs,outputs,global_batch_cnt,auc)

        

|2018-11-01 16:41:21| [e:01|b:0100] pos/neg:[1428/1644] logloss:[0.6539] auc:[0.6550] [111s]
|2018-11-01 16:43:13| [e:01|b:0200] pos/neg:[1447/1625] logloss:[0.6424] auc:[0.6780] [111s]
|2018-11-01 16:44:59| [e:01|b:0300] pos/neg:[1383/1689] logloss:[0.6395] auc:[0.6778] [106s]
|2018-11-01 16:46:49| [e:01|b:0400] pos/neg:[1439/1633] logloss:[0.6346] auc:[0.6928] [109s]
|2018-11-01 16:48:37| [e:01|b:0500] pos/neg:[1387/1685] logloss:[0.6280] auc:[0.6977] [107s]
|2018-11-01 16:50:22| [e:01|b:0600] pos/neg:[1353/1719] logloss:[0.6273] auc:[0.6950] [105s]
|2018-11-01 16:52:09| [e:01|b:0700] pos/neg:[1438/1634] logloss:[0.6380] auc:[0.6846] [107s]
|2018-11-01 16:53:55| [e:01|b:0800] pos/neg:[1401/1671] logloss:[0.6196] auc:[0.7120] [105s]
|2018-11-01 16:55:43| [e:01|b:0900] pos/neg:[1421/1651] logloss:[0.6211] auc:[0.7069] [108s]
|2018-11-01 16:57:30| [e:01|b:1000] pos/neg:[1427/1645] logloss:[0.6136] auc:[0.7214] [107s]
|2018-11-01 16:59:16| [e:01|b:1100] pos/neg:[1376/1696] logloss:[0.616

|2018-11-01 19:24:57| [e:07|b:0800] pos/neg:[1383/1689] logloss:[0.4680] auc:[0.8534] [108s]
|2018-11-01 19:26:44| [e:07|b:0900] pos/neg:[1391/1681] logloss:[0.4850] auc:[0.8369] [107s]
|2018-11-01 19:28:32| [e:07|b:1000] pos/neg:[1377/1695] logloss:[0.4727] auc:[0.8479] [108s]
|2018-11-01 19:30:21| [e:07|b:1100] pos/neg:[1347/1725] logloss:[0.4820] auc:[0.8446] [108s]
|2018-11-01 19:30:41| [e:07|b:1123] epoch-done
    valid_batch_cnt: [365] [0.21s/per]
|2018-11-01 19:35:46|  pos/neg: [461949/659331]
|2018-11-01 19:35:46| [e:07|b:1123 valid] valid_logloss:[0.74598] valid_auc:[0.66320]
|2018-11-01 19:37:35| [e:08|b:0100] pos/neg:[1371/1701] logloss:[0.4706] auc:[0.8491] [109s]
|2018-11-01 19:39:23| [e:08|b:0200] pos/neg:[1345/1727] logloss:[0.4673] auc:[0.8507] [107s]
|2018-11-01 19:41:13| [e:08|b:0300] pos/neg:[1475/1597] logloss:[0.4617] auc:[0.8605] [110s]
|2018-11-01 19:43:01| [e:08|b:0400] pos/neg:[1416/1656] logloss:[0.4661] auc:[0.8535] [107s]
|2018-11-01 19:44:48| [e:08|b:0500] 

KeyboardInterrupt: 

## Restore | 构造数据

In [ ]:
def make_data_ckpt():
    valid_input_dict = {"input/train_phase:0":False}
    label = None
    with tf.Session(graph=graph) as sess:
        sess.run(inp_iterator.initializer,valid_feed)
        tmpdict = sess.run(inp_next_dict)
    for k,v in tmpdict.items():
        if k=="label":
            label = v
        elif type(v) == tf.SparseTensorValue:
            valid_input_dict.update({"input/"+k+"/shape:0":v.dense_shape, "input/"+k+"/indices:0":v.indices, "input/"+k+"/values:0":v.values})
        else:
            valid_input_dict.update({"input/"+k+":0":v})
    return valid_input_dict,label

def make_data_pb():
    valid_input_dict = {"train_phase":False}
    label = None
    with tf.Session(graph=graph) as sess:
        sess.run(inp_iterator.initializer,valid_feed)
        tmpdict = sess.run(inp_next_dict)
    for k,v in tmpdict.items():
        if k=="label":
            label = v
        else:
            valid_input_dict.update({k:v})
    return valid_input_dict,label


## Restore | pb 
- ### simple_save API存的pb模型好像有问题，restore后auc0.5

In [ ]:
from tensorflow.python.saved_model import tag_constants
# model_p = "/home/zhoutong/tf_serve/midas_model/v3"
model_p = "/home/zhoutong/tf_serve/midas/model_2018-10-30-11-29-41/"
# model_p = "/home/zhoutong/starksdk_model/model_2018-10-25-19-56-17/model_of_auc-0.87386"

# valid_input_dict,label = make_data_pb()
valid_input_dict_ckpt,label = make_data_ckpt()


####################
# 方式一： 仍然用 graph  session回复模型然后预测,这个仍然需要feed_dict,用ckpt转成的pb
class Inference(object):
    def __init__(self,model_path,out_tensor_name="output/pred:0",inp_tensor_prefix="input"):
        # params
        self.model_p = model_path
        self.out_tensor_name= out_tensor_name
        self.inp_tensor_prefix = inp_tensor_prefix
        # graph & sess
        self.graph = tf.Graph()
        with self.graph.as_default():
            model_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
            model_config.gpu_options.allow_growth = True
            self.sess = tf.Session(graph=self.graph,config=model_config)
            # restore
            _ = tf.saved_model.loader.load(self.sess,[tag_constants.SERVING],self.model_p)
            init_op = tf.global_variables_initializer()
        # init
        self.sess.run(init_op)
        # prepare input & output
        self.pred = self.sess.graph.get_tensor_by_name(out_tensor_name)
        self.to_feed_ph = []
        for op in self.sess.graph.get_operations():
            if op.name.startswith(self.inp_tensor_prefix) and "label" not in op.name :
                ph = self.sess.graph.get_tensor_by_name(op.name+":0")
                self.to_feed_ph.append(ph)
        print("name of tensors(placeholder) to input:")
        for ph in self.to_feed_ph: 
            print("    ",ph.name)       
    
    def infer(self,inp_dict):
        feed_dict = {ph:inp_dict[ph.name] for ph in self.to_feed_ph}
        pred = self.sess.run(self.pred,feed_dict)
        return pred

infer_model = Inference(model_path=model_p)
y_pred = infer_model.infer(valid_input_dict_ckpt)
        
print("*"*50)
#####################
# 方式二：直接使用api， 不适用于ckpt转成的pb
# predict_fn = tf.contrib.predictor.from_saved_model(model_p)
# predictions = predict_fn(valid_input_dict)['pred']
# print("predictions:\n",predictions)
# print("label:\n",label)
# print("auc:\n",roc_auc_score(label,predictions))
# print("logloss:\n",log_loss(label, predictions))

print("y_pred:\n",y_pred)
print("label:\n",label)
if label is not None:
    print("auc:\n",roc_auc_score(label,y_pred))
    print("logloss:\n",log_loss(label, y_pred))
    pass

## Restore | ckpt

In [ ]:
valid_input_dict,label = make_data_ckpt()
base_path = "/home/zhoutong/midas_model/model_example"
base_path ="/home/zhoutong/starksdk_model/model_2018-10-26-14-05-09"
model_path = tf.train.latest_checkpoint(base_path) #  或者指定模型 +"/model.ckpt-562"

def inference_with_ckpt(valid_input_dict,model_p,label=None):
    g1 = tf.Graph()
    with g1.as_default():
        with tf.Session(graph=g1) as sess:
            saver = tf.train.import_meta_graph(model_p+".meta")
            saver.restore(sess, model_p)
            valid_placeholder_dict= {}
            for op in graph.get_operations():
                if op.name.startswith("input") and "label" not in op.name :
                    ph = sess.graph.get_tensor_by_name(op.name+":0")
                    valid_placeholder_dict.update({ph:valid_input_dict[op.name+":0"]})
            y_pred_op = sess.graph.get_tensor_by_name("output/pred:0")
            y_pred = sess.run(y_pred_op,valid_placeholder_dict)
            print("y_pred:\n",y_pred)
    if label is not None:
        loss = log_loss(label,y_pred,eps=1e-7)
        auc = roc_auc_score(label,y_pred)
        print(f"loss:[{loss:.4f}] auc:[{auc:.4f}]")
    pass

inference_with_ckpt(valid_input_dict,model_path,label)


## Resotre | ckpt -> pb

In [ ]:
# params
ckpt_model_dir = "/home/zhoutong/midas_model"+"/model_2018-10-30-11-29-41"
output_tensorName = "output/pred:0"
input_prefix = "input"
label_name = "label"

# load the model
checkpoint = tf.train.latest_checkpoint(ckpt_model_dir) # checkpoint = model_save_path + "/model.ckpt-562"

# tf_server directory
serve_path = "/home/zhoutong/tf_serve/{model_name}/{version}".format(model_name="midas_model",version=ckpt_model_dir.split("/")[-1])

# transforming
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph(checkpoint+".meta")
    graph = tf.get_default_graph()
    sess.run(tf.global_variables_initializer())
    # to save in model
    model_outputs = tf.saved_model.utils.build_tensor_info(graph.get_tensor_by_name(output_tensorName))
    model_inputs_dict = {}
    print("input op.names :")
    for op in graph.get_operations():
        if op.name.startswith(input_prefix) and label_name not in op.name :
            print("    ",op.name)
            ph = sess.graph.get_tensor_by_name(op.name+":0")
            model_inputs_dict.update({op.name+":0" : tf.saved_model.utils.build_tensor_info(ph)})
    # struct save model
    signature_definition = tf.saved_model.signature_def_utils.build_signature_def(inputs = model_inputs_dict,
                                                                                  outputs={'outputs':model_outputs},
                                                                                  method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME)
    signature_def_map = {tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:signature_definition}
    builder = tf.saved_model.builder.SavedModelBuilder(serve_path)
    builder.add_meta_graph_and_variables(sess,[tf.saved_model.tag_constants.SERVING],signature_def_map=signature_def_map)
    builder.save()
    print("model saved at: %s" % serve_path)

## Restore | pb -> onnx

In [ ]:
from tensorflow.core.framework import graph_pb2

from onnx_tf.common import get_output_node_names
from onnx_tf.frontend import tensorflow_graph_to_onnx_model
from google.protobuf import text_format

graph_def = graph_pb2.GraphDef()
with open("/home/zhoutong/tf_serve/midas_model/model_2018-10-30-11-29-41/saved_model.pb", "rb+") as f:   
    # load tf graph def
    proto_b = f.read()
    ?text_format.Merge
    text_format.Merge(proto_b, graph_def)
#     graph_def.ParseFromString(proto_b)
    
    
# get output node names
output = get_output_node_names(graph_def)

# convert tf graph to onnx model
model = tensorflow_graph_to_onnx_model(graph_def, output)
# with open("output_path", 'wb') as f:
#     f.write(model.SerializeToString())

In [ ]:
fileContent[:100]
fileContent[:100].strip()

## apus_ad | DeepFM | Estimator API

- ### train_func、eval_func -> input

In [ ]:
def train_input_fn():
    pass

def eval_input_fn():
    pass

- ### model_func -> classifier

In [ ]:
def deepfm_model_fn(features, labels, mode):
    # log
    if mode == tf.estimator.ModeKeys.PREDICT:
        tf.logging.info("my_model_fn: PREDICT, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.EVAL:
        tf.logging.info("my_model_fn: EVAL, {}".format(mode))
    elif mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info("my_model_fn: TRAIN, {}".format(mode))

    inp_list = features
    weights = _initialize_weights()
    with tf.name_scope("input"):
        train_phase = tf.placeholder(tf.bool,name="train_phase")
        dropout_keep_fm = tf.cond(train_phase,lambda:config.dropout_fm,lambda:[1.0]*len(config.dropout_fm))
        dropout_keep_deep = tf.cond(train_phase,lambda:config.dropout_deep,lambda:[1.0]*len(config.dropout_deep))
    deepfm_output = run_deepfm(config,weights,inp_list,train_phase,dropout_keep_fm,dropout_keep_deep)
    # prediction
    with tf.name_scope("output"):
        pred = tf.reshape(deepfm_output,[-1],name="pred")
    # logloss
    loss_op = tf.reduce_mean(tf.losses.log_loss(labels, pred))
    if config.l2_reg>0:
        loss_op += tf.contrib.layers.l2_regularizer(config.l2_reg)(weights["concat_projection"])
        for i in range(len(config.deep_layers)):
            loss_op += tf.contrib.layers.l2_regularizer(config.l2_reg)(weights["layer_%d"%i])
    # train_op
    _optimizer = tf.train.AdamOptimizer(learning_rate=config.learning_rate, beta1=0.9, beta2=0.999,epsilon=1e-8)
    grads = _optimizer.compute_gradients(loss)
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)
    for var in tf.trainable_variables():
        tf.summary.histogram(var.name, var)
    train_op = _optimizer.apply_gradients(grads)

    # tf.summary
    accuracy = tf.metrics.accuracy(labels, predictions=pred, name='acc_op')
    recall = tf.metrics.recall(labels, predictions=pred, name='recall_op')
    auc = tf.metrics.auc(labels, predictions=pred, name='auc_op')
    metrics = {'accuracy': accuracy, 'recall': recall, 'auc': auc}
    tf.summary.scalar('my_accuracy', accuracy[1])
    tf.summary.scalar('my_precision', precision[1])
    tf.summary.scalar('my_recall', recall[1])
    tf.summary.scalar('my_auc', auc[1])

    if mode==tf.estimator.ModeKeys.PREDICT:
        predictions = {"pred":pred}
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)
    if mode==tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
    if mode==tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


e_config = tf.estimator.RunConfig(save_checkpoints_steps=100,
                                  save_summary_steps=10,
                                  keep_checkpoint_max=3)
classifier = tf.estimator.Estimator(model_fn=deepfm_model_fn, 
                                    model_dir=config.model_dir+"/estimator",
                                    config=e_config)


- ### Traning:

In [ ]:
classifier.train(input_fn=lambda: train_inpu)

- ### Evaluation:

- ### Prediction:

## 测试 | FM部分是否正常（观察到embd不更新）

In [ ]:
gg = tf.Graph()
import shutil
with gg.as_default():
    # 需要： weights feat_total_idx_sp feat_total_value_sp
    dropout_keep_fm=[1.0,1.0]
    inp_tfrecord_path = tf.placeholder(dtype=tf.string, name="tfrecord_path")
    _optimizer = tf.train.AdamOptimizer(learning_rate=config.learning_rate, beta1=0.9, beta2=0.999,epsilon=1e-8)
    inp_iterator = get_input(inp_tfrecord_path)
    inp_next_dict = inp_iterator.get_next()
    feat_total_idx_sp,feat_total_value_sp = _get_total_feature(inp_next_dict)

    weights = _initialize_weights()
    y_first_order = tf.nn.embedding_lookup_sparse(
            weights["feature_bias"],
            sp_ids=feat_total_idx_sp,
            sp_weights=feat_total_value_sp,
            combiner="sum")

    y_first_order = tf.nn.dropout(
            y_first_order,
            dropout_keep_fm[0],
            name="y_first_order_dropout")


    # sum_square part
    summed_features_emb_square = tf.square(
        tf.nn.embedding_lookup_sparse(
            weights["feature_embeddings"],
            sp_ids=feat_total_idx_sp,
            sp_weights=feat_total_value_sp,
            combiner="sum"))
    # square_sum part
    squared_sum_features_emb = tf.nn.embedding_lookup_sparse(
        tf.square(weights["feature_embeddings"]),
        sp_ids=feat_total_idx_sp,
        sp_weights=tf.square(feat_total_value_sp),
        combiner="sum")
    # second order
    y_second_order = 0.5 * tf.subtract(
        summed_features_emb_square,
        squared_sum_features_emb)  # None * K
    y_second_order = tf.nn.dropout(y_second_order,
                                   dropout_keep_fm[1])  # None * K

    # label
    label = tf.cast(inp_next_dict['label'],dtype=tf.float32)
    # fm-way
    y_fm = tf.reshape(tf.reduce_sum(y_first_order+y_second_order,axis=1),[-1])
#     concat-way
    concat_input = tf.concat([y_first_order,y_second_order],axis=1)
    y_concat = tf.reshape(tf.matmul(concat_input, weights["concat_projection"])+weights['concat_bias'],[-1])
    
#     y_pred = tf.nn.sigmoid(y_fm)
    y_pred = tf.nn.sigmoid(y_concat)
    cost = tf.reduce_mean(tf.losses.log_loss(label, y_pred))
    opt=tf.train.AdamOptimizer(learning_rate=0.001)
    grad = opt.compute_gradients(cost)
    train_op=opt.apply_gradients(grad)
    feed_dict = {inp_tfrecord_path:config.train_tfrecord_file}

    # summary
    summary_save_path = "/home/zhoutong/tmp_summary"
    if os.path.exists(summary_save_path): shutil.rmtree(summary_save_path)
    _ = tf.summary.scalar("logloss",cost)
    for g,v in grad:
        if g is not None:
            _=tf.summary.histogram(v.op.name+"/gradients",g)
    for v in tf.trainable_variables():
        _=tf.summary.histogram(v.name.replace(":0","/value"),v)
    summary_op = tf.summary.merge_all()
    writer = tf.summary.FileWriter(summary_save_path, tf.get_default_graph())

with tf.Session(graph=gg) as sess:
    sess.run(tf.global_variables_initializer())
    epoch=10
    i = 0
    for _ in range(epoch):
        sess.run(inp_iterator.initializer,feed_dict)
        while True:
            i += 1
            try:
                to_run = [train_op,weights,cost,summary_op,y_first_order,y_second_order,y_fm]
                _,w,c,summary,y_1st,y_2nd,y_nosigmoid=sess.run(to_run,feed_dict,)
                writer.add_summary(summary,i)
                if (i % 20 ==0 and i < 200) or (i % 100==0 and i >=500):
                    print("*"*20,f"at range:{i}","*"*20)
                    c
            except tf.errors.OutOfRangeError:
                break
    

## 测试 | TFRecordDataset作为placeholder的输入
- 构造一个Dataset作为输入
- 因为TFRecordDataset传的是文件路径，意味着如果要预测的数据必须先存成一个文件

In [ ]:
test_dataset1 = tf.data.TFRecordDataset(valid_tfrecord_file,compression_type = compression_type)
test_ph = tf.placeholder()

## 测试 | 每个epoch重新加载dataset

In [ ]:
sess = tf.Session()

# sample = [[1,2,3] for _ in range(5)]
train = np.random.sample((10,3))
valid = np.random.sample((4,3))*10

train,valid
train_dataset = tf.data.Dataset.from_tensor_slices(train).batch(2)
valid_dataset = tf.data.Dataset.from_tensor_slices(valid).batch(2)
#################
# 第一种
# it = tf.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)
# train_init_op = it.make_initializer(train_dataset)
# valid_init_op = it.make_initializer(valid_dataset)
# epoch = 0
# for e in range(3):
#     epoch += 1
#     sess.run(train_init_op)
#     print("train-data:")
#     while True:
#         try:
#             sess.run(it.get_next())
#         except tf.errors.OutOfRangeError:
#             print("epcoh: %s train-data done" % epoch)
#             sess.run(valid_init_op)
#             print("valid-data:")
#             while True:
#                 try:
#                     sess.run(it.get_next())
#                 except tf.errors.OutOfRangeError:
#                     break
#             print("*"*40)
#             break



################
# 第二种
it_train = train_dataset.make_initializable_iterator()
it_valid = valid_dataset.make_initializable_iterator()
epoch = 0
for e in range(3):
    epoch += 1
    sess.run(it_train.initializer)
    print("train-data:")
    while True:
        try:
            sess.run(it_train.get_next())
        except tf.errors.OutOfRangeError:
            print("epcoh: %s train-data done" % epoch)
            sess.run(it_valid.initializer)
            print("valid-data:")
            while True:
                try:
                    sess.run(it_valid.get_next())
                except tf.errors.OutOfRangeError:
                    break
            print("*"*40)
            break
    pass

## 测试 | tf.cond

In [ ]:
a_graph = tf.Graph()
with a_graph.as_default():
    def _get_train_input():
        """
        目前预处理没有区分验证集和训练集
        """

        train = np.random.sample((10,3))
        train_dataset = tf.data.Dataset.from_tensor_slices(train).batch(2)
        iterator = train_dataset.make_initializable_iterator()
        return iterator

    # 验证集解析
    def _get_valid_input():
        """
        目前预处理没有区分验证集和训练集
        """
        valid = np.random.sample((10,3))*100
        valid_dataset = tf.data.Dataset.from_tensor_slices(valid).batch(2)
        iterator = valid_dataset.make_initializable_iterator()
        return iterator


    train_phase = tf.placeholder(dtype=tf.bool)
    tfrecord_path = tf.placeholder(dtype=tf.string)
    compression_type = tf.placeholder(dtype=tf.string)

    train_iterator = _get_train_input()
    valid_iterator = _get_valid_input()

    def f_true():
        result = train_iterator.get_next()
        return result
    def f_false():
        result = valid_iterator.get_next()
        return result

    inp_list = tf.cond(train_phase,f_true,f_false)

with tf.Session(graph=a_graph) as sess:
    feed_dict = {train_phase:True,tfrecord_path:tfrecord_file,compression_type:"GZIP"}
    sess.run(inp_iterator.initializer,feed_dict)
    sess.run(inp_list,feed_dict)


## 测试 | 两个稀疏向量的操作

In [ ]:
sess = tf.Session()

a = tf.SparseTensor(indices=[[0,1],[0,2]], values=[1,1], dense_shape=[5,3])
b = tf.SparseTensor(indices=[[1,5],[1,7],[1,8],[1,9]], values=[2,2,2,2], dense_shape=[5,7])
indices=b.indices
userindices = tf.where(tf.equal(indices[:, 0], 1))
print("a:")
sess.run(a)
print("b:")
sess.run(b)
print("concat:")
sess.run(tf.sparse_concat(axis=1,sp_inputs=[a,b]))

# sess.run(tf.sparse_add(a,b)) # shape不同不能add



# a_add_b = tf.sparse_add(a,b)
# indices = a_add_b.indices
# values = a_add_b.values
# count_list = tf.unique_with_counts(indices[:, 0]).count
# count_list_ = tf.concat([[0],count_list],axis=0)[:-1]
# count_result = tf.transpose(tf.stack([count_list_,count_list]))
# print("count:")
# sess.run(count_result)
# print("result:")
# sess.run(tf.map_fn(lambda x: values[x[0]:x[1]], count_result))
# print("userindices")
# sess.run(userindices)
# sess.run(tf.gather(indices,userindices))
# print("gather value")
# sess.run(tf.gather(values,userindices))
# sess.run(userindices)

## 测试 | 构造行号 | tf.tile或者pyfunc

In [ ]:
sess = tf.Session()
ff = tf.constant([[833,119,8922,333],[837,119,8999,330]])

print("ff:")
sess.run(ff)

############
# tf.tile

idx = tf.range(tf.shape(ff)[0])
idx_2d = tf.Variable(tf.reshape(idx,[-1,1]))
idx_2d_full = tf.tile(idx_2d,[1,tf.shape(ff)[1]])
result = tf.concat([tf.reshape(idx_2d_full,[-1,1]),tf.reshape(ff,[-1,1])],axis=1)
sess.run(tf.global_variables_initializer())
print("idx:")
sess.run(idx_2d)
sess.run(idx_2d_full)
print("tf.tile result:")
sess.run(result)

############
# pyfunc
def add_idx(arr):
    result = []
    for i in range(len(arr)):
        for j in arr[i]:
            result.append([i, j])
    return np.array(result)
result_1 = tf.py_func(func=add_idx, inp=[ff],Tout=[tf.int64])
print("py_func_result1:")
sess.run(result_1)



In [ ]:
import functools

def wrap_func(ori_func,new_func):
    @functools.wraps(ori_func)
    def run(*args, **kwargs):
        return new_func(ori_func, *args, **kwargs)
    return run
def replaced_print(ori_function, parameter):
    now = time.strftime("%Y-%m-%d %H:%M", time.localtime(time.time()))
    new_params = now+": "+parameter
    return ori_function(new_params)
# old_print = print
print=wrap_func(print,replaced_print)
print("print方法已经被hook,会自动输出时间前缀")


In [65]:
import tf2onnx
with tf.Session() as sess:
    x = tf.placeholder(tf.float32, [2, 3], name="input")
    x_ = tf.add(x, x)
    _ = tf.identity(x_, name="output")
    onnx_graph = tf2onnx.tfonnx.process_tf_graph(sess.graph)
    model_proto = onnx_graph.make_model("test", ["input:0"], ["output:0"])
    with open("/home/zhoutong/tmp/model.onnx", "wb") as f:
        f.write(model_proto.SerializeToString())

ModuleNotFoundError: No module named 'tf2onnx'